# Libraries

In [1]:
import numpy as np
from omegaconf import OmegaConf
import models
import loss_func
import train
import loader

# Load Data

In [2]:
cfg = OmegaConf.load("config.yaml")
cmd_cfg = OmegaConf.from_cli()
cfg = OmegaConf.merge(cfg, cmd_cfg)
print(OmegaConf.to_yaml(cfg))

seed: 42
optimize:
  optimizer: Adam
  lr: 0.001
train:
  epochs: 100
  batch_size: 32
dataset:
  path: MNIST
  test_size: 0.2
  train_size: 0.6
  val_size: 0.2
  num_workers: 2
loss: MSE
--ip: 127.0.0.1
--stdin: 9008
--control: 9006
--hb: 9005
--Session:
  signature_scheme: hmac-sha256
  key: b"9832398d-75e8-4677-a650-99a5204a2628"
--shell: 9007
--transport: tcp
--iopub: 9009
--f: c:\Users\aleja\AppData\Roaming\jupyter\runtime\kernel-v2-24160Zv12T1wKSz1V.json



In [3]:
timeseries = np.random.rand(10, 1000)

train_loader, val_loader, test_loader = loader.generateLoaders(timeseries, 'pearson')

X_train shape:  (10, 600)
X_val shape:  (10, 200)
X_test shape:  (10, 200)


In [4]:
window = 20
model = models.Net(window, 1, 32)

lossfn = loss_func.getLoss()

import torch

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model, train_losses, val_losses, accuracy_list  = train.train(model, train_loader, val_loader, optimizer)

Using device: cuda
cuda
epoch: 10 	 training loss: 0.08755496069788933 	 validation loss: 0.0880552480618159 	 accuracy : 1.0
epoch: 20 	 training loss: 0.085481396317482 	 validation loss: 0.08557697385549545 	 accuracy : 1.0
epoch: 30 	 training loss: 0.0812800832092762 	 validation loss: 0.08511076122522354 	 accuracy : 1.0
epoch: 40 	 training loss: 0.08101287335157395 	 validation loss: 0.08495927353700002 	 accuracy : 1.0
epoch: 50 	 training loss: 0.08152497634291649 	 validation loss: 0.0846748302380244 	 accuracy : 1.0
epoch: 60 	 training loss: 0.08089155182242394 	 validation loss: 0.08488558729489644 	 accuracy : 1.0
epoch: 70 	 training loss: 0.0828028954565525 	 validation loss: 0.08458059281110764 	 accuracy : 1.0
epoch: 80 	 training loss: 0.08095723986625672 	 validation loss: 0.08521700650453568 	 accuracy : 1.0
epoch: 90 	 training loss: 0.07851542457938195 	 validation loss: 0.08550844093163808 	 accuracy : 1.0
epoch: 100 	 training loss: 0.08145254403352738 	 valid